In [8]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [10]:
from utils import sample_data

NB_SAMPLES = 100
PATH_SAMPLE = Path("../data/samples/sample_{}_train.csv".format(NB_SAMPLES))
DATA_SAMPLE = pd.read_csv(PATH_SAMPLE)

In [13]:
print(DATA_SAMPLE)

        textID                                               text  \
0   c46ff87273   why, hello there  How`s life on the other sid...   
1   020b321aec                         are you call that winning?   
2   079e4e3472   or even NOOOOO NOT THE SECRET NAMEREBECCA PLEASE   
3   c5637a0ed2  Bran!! Watch my video!!!!! please dont make me...   
4   d5e35a43d6  Ahh feels good to have played guitar hero afte...   
..         ...                                                ...   
94  2e96695013              morning michelle  how`s the oatmeal ?   
95  8e5f9c7b1e   Well I will send out an email and you all can...   
96  01e1f59fac  cant believe i stay out this late!.....waking ...   
97  03e8d4538e                   Oh, I`m jealous (how surprising)   
98  c5ec1e7875               are you and melissa celebrities now?   

                                        selected_text  sentiment  
0   why, hello there  How`s life on the other side...          0  
1                          are you ca

# Pre-processing

In [ ]:
from descriptor.glove_descriptor.glove_descriptor import descriptor

WORD_SIZE = 50  # 50 or 100 or 200 or 300
SENTENCE_SIZE = 50  # What ever
FILL_WITH = 0  # Should be an integer
SPLIT_PUNCTUATION = False # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not

(X_STRING, X_SCALAR) = descriptor(DATA_SAMPLE["text"], DESCRIPTIONS, SENTENCE_SIZE, WORD_SIZE, FILL_WITH)

print(DATA.shape, X_STRING.shape, Y.shape, X_SCALAR.shape)

print("\nOriginal data :")
print(DATA[0])
print("\nFilled sentence :")
print(X_STRING[0])
print("\nLabel :")
print(Y[0])
print("\nDescriptor :")
print(X_SCALAR[0])

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nb_neighbors = 2

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(X_SCALAR, Y)

In [ ]:
from utils.post_processing import pred_to_sentence, sentence_to_string, filter_character

pred = knn.predict(X_SCALAR)
meaning_sentences = pred_to_sentence(X_STRING, pred)

results = []
for sentence in meaning_sentences:
    result = ""
    for word in sentence:
        filtered_word = filter_character(word, "$")
        if len(filtered_word) != 0:
            result += filtered_word + " "
    results.append(result)
results=np.array(results, dtype=object)

In [ ]:
print(results)

In [ ]:
print(DATA[:, 2])

In [ ]:
from utils.loss import jaccard

avg = 0
for i in range(len(results)):
    print(avg)
    avg += jaccard(results[i], SMALL_DATA.sample_data["selected_text"].to_numpy()[i])
avg /= len(results)

print(avg)